In [1]:
from tensorflow.keras.optimizers import Adam
from keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
from eval_utils import calculate_results, prepare_for_evaluation, generate_report, encode_sequences, encode_output
from dataloader import *
from  config_translator import *
from data_processor import preprocess_data
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

ModuleNotFoundError: No module named 'keras'

In [ ]:
data = DataLoader(config_mixed_coco14_coco14_glove)

In [ ]:
data=preprocess_data(data)

In [ ]:

def data_generator(train_input, input_tokenizer, max_input_length, train_output, output_tokenizer, max_output_length,
                   output_vocab_size, batch_size=64):
    while 1:
        count = 0

        while 1:
            if count >= len(train_output):
                count = 0

            input_seq = list()
            output_seq = list()
            for i in range(count, min(len(train_input), count+batch_size)):
                input, output = train_input[i], train_output[i]
                input_seq.append(input)
                output_seq.append(output)
            input_seq = encode_sequences(input_tokenizer, max_input_length, input_seq)
            output_seq = encode_sequences(output_tokenizer, max_output_length, output_seq)
            output_seq = encode_output(output_seq, output_vocab_size)

            count = count + batch_size

            input_seq = np.array(input_seq)
            output_seq = np.array(output_seq)
            output_seq = output_seq.reshape((output_seq.shape[0], output_seq.shape[1],output_vocab_size))
            import tensorflow as tf
            with tf.device('/cpu:0'):
                input_seq = tf.convert_to_tensor(input_seq, np.uint8)
                output_seq = tf.convert_to_tensor(output_seq, np.uint8)
            yield [input_seq, output_seq]

class ModelImpl:
    def __init__(self, data):
        self.data=data
        def define_model_glove(input_vocab_size, output_vocab_size, max_input_length, max_output_length, n_units,
                               embedding_matrix, embedings_dim):
            model = Sequential()

            emb = Embedding(input_vocab_size, embedings_dim, input_length=max_input_length, weights=[embedding_matrix], mask_zero=True)
            model.add(emb)
            lstm_lay = LSTM(n_units)
            model.add(lstm_lay)
            model.add(RepeatVector(max_output_length))
            model.add(LSTM(n_units, return_sequences=True))
            model.add(TimeDistributed(Dense(output_vocab_size, activation='softmax')))
            return model

        self.model = define_model_glove(self.data.input_vocab_size, self.data.output_vocab_size, self.data.max_input_length,
                           self.data.max_output_length, 256, self.data.embedding_matrix_input, glove["embedings_dim"])
        self.model.compile(
            optimizer=self.optimizer(),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        self.model.summary()
        self.setup()

    def setup(self):
        # model.optimizer.lr = 0.001
        self.epochs = 100
        self.batch_size=200
        self.steps = len(self.data.train_output_sentences_list) // self.batch_size

    def optimizer(self):
        return Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    def train(self):
        model_weights_path="./" + self.data.configuration["data_name"] + self.data.configuration["model_save_dir"]
        if self.data.configuration["train_model"]:
            es = EarlyStopping(monitor='loss', min_delta=0.001, patience=3)
            filepath = model_weights_path + self.data.configuration["model_save_path"]
            checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True,
                                         mode='min', save_weights_only=False)
            callbacks_list = [checkpoint, es, CSVLogger("./" + self.data.configuration["data_name"] +'/logs.csv', separator=",", append=True),]
            if self.data.configuration["continue_training"]:
                self.model = load_model(filepath)
                print("New model loaded")

            train_generator = data_generator(self.data.train_bbox_categories_list, self.data.input_tokenizer,
                                             self.data.max_input_length,self.data.train_output_sentences_list,
                                             self.data.output_tokenizer,self.data.max_output_length,
                                             self.data.output_vocab_size, batch_size= self.batch_size)
            self.model.fit(train_generator,
                epochs=self.epochs,
                steps_per_epoch=self.steps,
                callbacks=[callbacks_list],
                verbose=1,
            )
            if self.data.configuration["save_model"]:
                writepath = model_weights_path+ "/"+'model' + '.h5'
                self.model.save(writepath)
                self.model.save_weights(model_weights_path
                                        + self.data.configuration["model_save_path"])
        else:
            self.model.load_weights(model_weights_path
                                        +self.data.configuration["model_save_path"])

    def evaluate(self):
        expected, results = prepare_for_evaluation(self.data.encoder_test_sequences, self.data.test_captions_mapping,
                                                   model, self.model, self.data.word2idx_outputs
                                                   , self.data.max_output_len)
        out = calculate_results(expected, results, self.data.configuration)
        print(out)
model=ModelImpl(data)

In [ ]:
model.train()

In [ ]:
model.evaluate()

In [ ]:
generate_report(general["results_directory"])